
# <a id='toc1_'></a>[Consolidate BioFIND clinical data and alpha-synuclein SAA data](#toc0_)

Notebook to tidy and consolidate multiple BioFIND data files into single table to facilitate application of new research diagnostic/staging system to BioFIND participants.

**Date:** 11/3/2024  
**Author:** Marco Russo, MD, PhD 
https://github.com/dr-russo    

**Libraries**
* pandas  
* numpy   

**BioFIND Data Files**  
Data  obtained from the Fox Investigation for New Discovery of Biomarkers  (BioFIND) database, which is part of the USC Laboratory of Neuro Imaging (LONI) Image & Data Archive (IDA), through limited Data Use Agreement: https://ida.loni.usc.edu/collaboration/access/appLicense.jsp. For up-to-date information on the study, visit [www.michaeljfox.org/biofind](www.michaeljfox.org/biofind).

Note, upon download, .csv file names have date of download appended as _%d%b%Y format, so adjust filenames accordingly:
* Primary_Diagnosis_23Mar2024.csv | PATNO, PRIMMDIAG
* Screening_Demographics_23Mar2024.csv | PATNO, 
* MDS_UPDRS_Part_I__Patient_Questionnaire_23Mar2024.csv
* MDS_UPDRS_Part_I_23Mar2024.csv
* MDS_UPDRS_Part_II__Patient_Questionnaire_23Mar2024.csv
* MDS_UPDRS_Part_III__Post_Dose__23Mar2024.csv
* MDS_UPDRS_Part_IV_23Mar2024.csv 
* Use_of_PD_Medication_23Mar2024.csv
* REM_Sleep_Disorder_Questionnaire_23Mar2024.csv
* Montreal_Cognitive_Assessment__MoCA__23Mar2024.csv
* Biospecimen_Analysis_Results_23Mar2024.csv
* PD_Features_23Mar2024.csv

**Derived Data Files**  
* biofind_ledd.csv  - created by biofind_ledd.ipynb

**Output**
* biofind_mds-updrs_data.csv - consolidated data table for all MDS-UPDRS data



**Abbreviations**  
PD: Parkinson's disease
BioFIND: Fox Investigation for New Discovery of Biomarkers (BioFIND) cohort study
LEDD: levdopa-equivalent daily dose
MDS-UPDRS: Movement Disorder Society Unified Parkinson’s Disease Rating Scale
MoCA: Montreal Cognitive Assessment
NSD: Neuronal Synuclein Disease
ISS: Integrated Staging System
RBD: REM sleep behavior disorder


**Table of contents**<a id='toc0_'></a>    
- [Consolidate BioFIND clinical data and alpha-synuclein SAA data](#toc1_)    
    - [Import list of participants and begin merging other data tables](#toc1_1_1_)    
    - [Consolidate all MDS-UPDRS data](#toc1_1_2_)    
    - [MDS-UPDRS Part 3 ON and OFF Medication Scores](#toc1_1_3_)    
    - [Additional Non-motor Metrics](#toc1_1_4_)    
    - [Combine all data tables into master table of BioFIND clinical and aSyn-SAA data](#toc1_1_5_)    
    - [Write composite data to file](#toc1_1_6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
# Imports
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

In [ ]:
# Set display options
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)

# Enable multiline outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'     

# Set float formatting
from IPython.display import display
pd.options.display.float_format = '{:,.2f}'.format

### <a id='toc1_1_1_'></a>[Import list of participants and begin merging other data tables](#toc0_)

In [3]:
# Start with primary diagnosis table as master list of consented paricipants 
# PATNO is index
data = pd.read_csv('Primary_Diagnosis_23Mar2024.csv', header=0, index_col='PATNO')
data.sort_index().head() 

,REC_ID,F_STATUS,EVENT_ID,PAG_NAME,INFODT,PRIMDIAG,OTHNEURO,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
PATNO,,,,,,,,,,,
1001,386165501,S,BL,PRIMDXPD,02/2013,17,NaN,02/2013,2016-01-19 10:37:07.0,NaN,04/2013
1002,388148401,S,BL,PRIMDXPD,03/2013,1,NaN,03/2013,2016-01-19 10:37:08.0,NaN,04/2013
1003,391865801,S,BL,PRIMDXPD,03/2013,1,NaN,04/2013,2016-01-19 10:37:09.0,NaN,04/2013
1004,392462501,S,BL,PRIMDXPD,04/2013,1,NaN,04/2013,2016-01-19 10:37:10.0,NaN,04/2013
1005,398569401,S,BL,PRIMDXPD,05/2013,17,NaN,05/2013,2016-01-19 10:37:12.0,NaN,05/2013


In [4]:
# Load Screening/Demographics table  
screen = pd.read_csv('Screening_Demographics_23Mar2024.csv', header=0, index_col='PATNO')

In [5]:
# Merge screen into initial data table on PATNO
data = pd.merge(left=data, right=screen[['BIRTHDT','APPRDX','PRJENRDT','EXCLUDED','GENDER']], left_index=True, right_index=True, how='left')

In [6]:
# Convert to datetime types
data['BIRTHDT'] = pd.to_datetime(data['BIRTHDT'], format='%Y', yearfirst=True)
data['INFODT'] = pd.to_datetime(data['INFODT'], format='mixed')
data['PRJENRDT'] = pd.to_datetime(data['PRJENRDT'], format='mixed')

# Calculate AGE at enrollment
data['AGE'] = ((data['PRJENRDT'] - data['BIRTHDT']) / np.timedelta64(365,'D')).round(1)

In [7]:
# Include only non-excluded subjects - NaN for excluded => enrolled
data = data[data['EXCLUDED'].isnull()]

In [8]:
# List of standard columns to drop prior to merges (to avoid naming conflicts and subscripts appended to column names)
drop_cols = ['REC_ID','F_STATUS','PAG_NAME','ORIG_ENTRY','LAST_UPDATE','QUERY','SITE_APRV']
data.drop(columns=drop_cols, inplace=True)

In [9]:
# Select only PRIMDIAG == 1 for PD participants
# Select only baseline (BL) visits 
pd_subgroup = data[(data['PRIMDIAG'] == 1)].where(data['EVENT_ID'] == 'BL')
pd_subgroup.drop(columns=['INFODT','OTHNEURO'], inplace=True)

# Check, n = 119 participants 
pd_subgroup.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 119 entries, 1002 to 1200
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   EVENT_ID  119 non-null    object        
 1   PRIMDIAG  119 non-null    int64         
 2   BIRTHDT   119 non-null    datetime64[ns]
 3   APPRDX    119 non-null    int64         
 4   PRJENRDT  119 non-null    datetime64[ns]
 5   EXCLUDED  0 non-null      float64       
 6   GENDER    119 non-null    int64         
 7   AGE       119 non-null    float64       
dtypes: datetime64[ns](2), float64(2), int64(3), object(1)
memory usage: 12.4+ KB


In [10]:
# Read data table with medication status
med_status = pd.read_csv('Use_of_PD_Medication_23Mar2024.csv', header=0, index_col='PATNO')

In [11]:
# Merge medication status into pd_subgroup, take only 'BL' data
med_status = med_status[med_status['EVENT_ID'] == 'BL']
pd_subgroup =  pd.merge(left=pd_subgroup, right=med_status['PDMEDYN'], left_index=True, right_index=True, how='left')
pd_subgroup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119 entries, 1002 to 1200
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   EVENT_ID  119 non-null    object        
 1   PRIMDIAG  119 non-null    int64         
 2   BIRTHDT   119 non-null    datetime64[ns]
 3   APPRDX    119 non-null    int64         
 4   PRJENRDT  119 non-null    datetime64[ns]
 5   EXCLUDED  0 non-null      float64       
 6   GENDER    119 non-null    int64         
 7   AGE       119 non-null    float64       
 8   PDMEDYN   119 non-null    int64         
dtypes: datetime64[ns](2), float64(2), int64(4), object(1)
memory usage: 13.4+ KB


In [12]:
# Load PD features table 
# Provides times since diagnosis and times since diagnosis
pd_features = pd.read_csv('PD_Features_23Mar2024.csv', dtype={'SXMO':'Int64', 'SXYEAR':'Int64'}, header=0, index_col='PATNO')

In [13]:
# Fill several NaN values in SXMO
pd_features.fillna({'SXMO': 6}, inplace=True) 

In [14]:
# Combine time of symptom onset into single month-year format
pd_features['SXDT'] = pd.to_datetime(pd_features['SXMO'].astype(str) + '/' + pd_features['SXYEAR'].astype(str), format='mixed')

In [15]:
# Convert to datetime format
pd_features['INFODT'] = pd.to_datetime(pd_features['INFODT'], format='mixed')
pd_features['PDDXDT'] = pd.to_datetime(pd_features['PDDXDT'], format='mixed')

In [16]:
# Calculate disease duration from symptom onset 
pd_features['DUR_SYMP'] = (pd_features['INFODT'] - pd_features['SXDT']) / np.timedelta64(365, 'D')

In [17]:
# Calculate disease duration from initial diagnosis
pd_features['DUR_DIAG'] = (pd_features['INFODT'] - pd_features['PDDXDT']) / np.timedelta64(365, 'D')

pd_features.loc[pd_features['DUR_DIAG'] < 0, 'DUR_DIAG'] = 0  # Coerce negative values to 0 - due to PATNO 1077 where INFODT 4/2014 but PDDXDT 6/2014, only instance

In [ ]:
# Uncomment these to see deccriptive statistics for Time since Diagnosis and Time since Symptom onset
# pd_features['DUR_DIAG'].apply([np.mean, np.median, np.std, np.min, np.max])
# pd_features['DUR_SYMP'].apply([np.mean, np.median, np.std, np.min, np.max])

### <a id='toc1_1_2_'></a>[Consolidate all MDS-UPDRS data](#toc0_)

In [20]:
# Load UPDRS Parts 1, 1Q, 2, 3
part1 = pd.read_csv('MDS_UPDRS_Part_I_23Mar2024.csv', header=0, index_col='PATNO')
part1q = pd.read_csv('MDS_UPDRS_Part_I__Patient_Questionnaire_23Mar2024.csv', header=0, index_col='PATNO')
part2 = pd.read_csv('MDS_UPDRS_Part_II__Patient_Questionnaire_23Mar2024.csv', header=0, index_col='PATNO')
part3 = pd.read_csv('MDS_UPDRS_Part_III__Post_Dose__23Mar2024.csv', header=0, index_col='PATNO')

In [22]:
# Create several lists of column names to easily reference appropriate items from each MDS-UPDRS part

# Create lists of columns to be included from each csv file
part1_cols = ["NP1COG", "NP1HALL", "NP1DPRS", "NP1ANXS", "NP1APAT", "NP1DDS"]
part1q_cols = [
    "NP1SLPN",
    "NP1SLPD",
    "NP1PAIN",
    "NP1URIN",
    "NP1CNST",
    "NP1LTHD",
    "NP1FATG",
]

# Remove Q1.1 or NP1COG from TOTAL SCORE for PART I
part1_cols_scoring = [
    "NP1HALL",
    "NP1DPRS",
    "NP1ANXS",
    "NP1APAT",
    "NP1DDS",
    "NP1SLPN",
    "NP1SLPD",
    "NP1PAIN",
    "NP1URIN",
    "NP1CNST",
    "NP1LTHD",
    "NP1FATG",
]

part2_cols = [
    "NP2SPCH",
    "NP2SALV",
    "NP2SWAL",
    "NP2EAT",
    "NP2DRES",
    "NP2HYGN",
    "NP2HWRT",
    "NP2HOBB",
    "NP2TURN",
    "NP2TRMR",
    "NP2RISE",
    "NP2WALK",
    "NP2FREZ",
]

part3_cols = [
    "NP3SPCH",
    "NP3FACXP",
    "NP3RIGN",
    "NP3RIGRU",
    "NP3RIGLU",
    "PN3RIGRL",
    "NP3RIGLL",
    "NP3FTAPR",
    "NP3FTAPL",
    "NP3HMOVR",
    "NP3HMOVL",
    "NP3PRSPR",
    "NP3PRSPL",
    "NP3TTAPR",
    "NP3TTAPL",
    "NP3LGAGR",
    "NP3LGAGL",
    "NP3RISNG",
    "NP3GAIT",
    "NP3FRZGT",
    "NP3PSTBL",
    "NP3POSTR",
    "NP3BRADY",
    "NP3PTRMR",
    "NP3PTRML",
    "NP3KTRMR",
    "NP3KTRML",
    "NP3RTARU",
    "NP3RTALU",
    "NP3RTARL",
    "NP3RTALL",
    "NP3RTALJ",
    "NP3RTCON",
    "DYSKPRES",
    "DYSKIRAT",
    "NHY",
]

# Remove postural and kinetic/action tremor from UPDRS PART III scoring
part3_cols_scoring = [
    "NP3SPCH",
    "NP3FACXP",
    "NP3RIGN",
    "NP3RIGRU",
    "NP3RIGLU",
    "PN3RIGRL",
    "NP3RIGLL",
    "NP3FTAPR",
    "NP3FTAPL",
    "NP3HMOVR",
    "NP3HMOVL",
    "NP3PRSPR",
    "NP3PRSPL",
    "NP3TTAPR",
    "NP3TTAPL",
    "NP3LGAGR",
    "NP3LGAGL",
    "NP3RISNG",
    "NP3GAIT",
    "NP3FRZGT",
    "NP3PSTBL",
    "NP3POSTR",
    "NP3BRADY",
    "NP3RTARU",
    "NP3RTALU",
    "NP3RTARL",
    "NP3RTALL",
    "NP3RTALJ",
    "NP3RTCON",
]

# part4_cols = ['NP4WDYSK','NP4DYSKI','NP4OFF','NP4FLCTI','NP4FLCTX','NP4DYSTN'] --NOT NEEDED

# TD columns
td_score_cols = [
    "NP2TRMR",
    "NP3PTRMR",
    "NP3PTRML",
    "NP3KTRMR",
    "NP3KTRML",
    "NP3RTARU",
    "NP3RTALU",
    "NP3RTARL",
    "NP3RTALL",
    "NP3RTALJ",
    "NP3RTCON",
]

pigd_score_cols = ["NP2WALK", "NP2FREZ", "NP3GAIT", "NP3FRZGT", "NP3PSTBL"]

In [ ]:
# Merge UPDRS Parts into single master MDS-UPDRS table using the above lists to select only needed columns

# MDS-UPRDS Part 1
part1 = part1[part1['EVENT_ID']=='BL'] # Baseline only data
updrs_data = pd.merge(left=pd_subgroup, right=part1[part1_cols], how='left', left_index=True, right_index=True, validate='1:1')

In [24]:
# MDS-UPDRS Part 1Q
part1q = part1q[part1q['EVENT_ID'] == 'BL'] # Baseline only data
updrs_data = pd.merge(left=updrs_data, right=part1q[part1q_cols], how='left',left_index=True, right_index=True)

In [25]:
# MDS-UPDRS Part 2
part2 = part2[part2['EVENT_ID']=='BL']
updrs_data = pd.merge(left=updrs_data, right=part2[part2_cols], how='left',left_index=True, right_index=True)

In [29]:
part3 = part3[part3['EVENT_ID']=='BL']
updrs_data = pd.merge(left=updrs_data, right=part3[part3_cols], left_index=True, right_index=True, how='left')

In [26]:
# MDS-UPDRS Part 3 Totals
part3['P3TOT'] = part3[part3_cols_scoring].sum(axis=1)

In [ ]:
updrs_data['P1TOT'] = updrs_data[part1_cols_scoring].sum(axis=1) 
updrs_data['P2TOT'] = updrs_data[part2_cols].sum(axis=1)
updrs_data['P3TOT'] = updrs_data[part3_cols_scoring].sum(axis=1) # Effectively 'ON' scores since taken from baseline

### <a id='toc1_1_3_'></a>[MDS-UPDRS Part 3 ON and OFF Medication Scores](#toc0_)

In [27]:
# MDS-UPDRS Part 3
# Part 3 unique as it has both OFF MEDICATION and ON MEDICATION scores 
part3_onoff = part3.pivot_table(index='PATNO',values='P3TOT',columns='EVENT_ID')
part3_onoff.rename(columns={'BL':'P3ON','V02':'P3OFF'}, inplace=True)

In [31]:
updrs_data = pd.merge(left=updrs_data, right=part3_onoff[['P3ON','P3OFF']], left_index=True, right_index=True, how='left')

In [ ]:
# Write UPDRS composite data to separate file
updrs_data.to_csv('biofind_mds-updrs_data.csv')

### <a id='toc1_1_4_'></a>[Additional Non-motor Metrics](#toc0_)

In [33]:
# Create list of relevant columns from RBD sleep questionnaire 
rbd_cols = ['DRMVIVID','DRMAGRAC','DRMNOCTB','SLPLMBMV','SLPINJUR','DRMVERBL','DRMFIGHT','DRMUMV','DRMOBJFL','MVAWAKEN','DRMREMEM','SLPDSTRB']
rbd_other_cols = ['STROKE','HETRA','PARKISM','RLS','NARCLPSY','DEPRS','EPILEPSY','BRNINFM','CNSOTH']

In [40]:
# Load RBD Sleep Questionnaire 
rbdsq_data = pd.read_csv('REM_Sleep_Disorder_Questionnaire_23Mar2024.csv', index_col='PATNO', header=0)

In [41]:
# Select only baseline data 
rbdsq_data = rbdsq[rbdsq['EVENT_ID']=='BL']

In [42]:
# Calculate total RBDSQ score from sum of questions 1-10, with single point for any positive response for Q10 [as in BioFIND]
rbdsq_data['RBDSQ_TOTAL'] = rbdsq_data[rbd_cols].sum(axis=1) + rbdsq_data[rbd_other_cols].any(axis=1)

In [43]:
# Flag RBD status as POSITIVE by questionnaire if Total RBD score >= 6
rbdsq_data['RBD_STATUS'] = rbdsq_data['RBDSQ_TOTAL'].apply(lambda x : 1 if x >= 6 else 0)

In [44]:
# Load Montreal Cognitive Assessment (MoCA) data 
moca_data = pd.read_csv('Montreal_Cognitive_Assessment__MoCA__23Mar2024.csv', header=0, index_col='PATNO')

In [45]:
# Load biospecimen data - for seed amplification assay (SAA) data
biospec_data = pd.read_csv('Biospecimen_Analysis_Results_23Mar2024.csv', header=0)

In [46]:
# Alpha-Syn Seed Amplification Assay (SAA) data (3 labs)
saa_data = biospec_data[biospec_data['TESTNAME'].isin(['RT-QuIC Result','RT-QuIC','aSyn-PMCA'])].pivot(index='PATNO',columns='PI_NAME',values='TESTVALUE')

In [ ]:
# Load LEDD medication data that is derived from accompanying notebook 'biofind_ledd.ipynb'
ledd_data = pd.read_csv('biofind_ledd.csv', index_col='PATNO', header=0)

### <a id='toc1_1_5_'></a>[Combine all data tables into master table of BioFIND clinical and aSyn-SAA data](#toc0_)

In [48]:
# Combine into master data table that includes MED STATUS, UPDRS, RBDSQ, and MOCA
data = pd.merge(left=updrs_data, right=rbdsq_data[['RBDSQ_TOTAL','RBD_STATUS']], left_index=True, right_index=True, how='left')

In [49]:
data = pd.merge(left=data, right=moca_data[['MCATOT']], left_index=True, right_index=True, how='left')

In [50]:
data = pd.merge(left=data, right=saa_data, left_index=True, right_index=True, how='left')

In [51]:
data = pd.merge(left=data, right=pd_features[['DUR_SYMP', 'DUR_DIAG']], left_index=True, right_index=True, how='left')

In [53]:
data = pd.merge(left=data, right=ledd_data, left_index=True, right_index=True, how='left')

In [54]:
# Rename aSyn-SAA results according to each lab PI in simplified form
data.rename(columns={'Alison Green':'SAA_AG','Byron Caughey':'SAA_BC', 'Claudio Soto':'SAA_CS'}, inplace=True)

In [55]:
# Calculate TD score as sum of td_score_cols in MDS-UPRDS Part 3 
data['TDSCORE'] = data[td_score_cols].sum(axis=1)

In [56]:
# Calculate PIGD score as sum of td_score_cols in MDS-UPRDS Part 3 
data['PIGDSCORE'] = data[pigd_score_cols].sum(axis=1)

In [57]:
# Calculate ratio of TD score to PIGD score
data['TDPIGDRATIO'] = data['TDSCORE']/data['PIGDSCORE']
data.loc[~np.isfinite(data['TDPIGDRATIO']),'TDPIGDRATIO'] = np.nan

In [ ]:
# Determine the motor phenotype TD vs. PIGD according to the following comparisons:
def calc_tdpigd(ratio):
    if ratio >= 1.15:
        return 'TD'
    elif ratio <= 0.9:
        return 'PIGD'
    elif (ratio > 0.9) & (ratio < 1.15):
        return 'IND'

data.loc[:,'MOTORTYPE'] = data['TDPIGDRATIO'].apply(lambda x: calc_tdpigd(x))

### <a id='toc1_1_6_'></a>[Write composite data to file](#toc0_)

In [59]:
# Save composite data, includes: 
# PD Features
# Demographics
# MDS-UPDRS
# RBDQ
# MoCA 
# aSyn-SAA data
data.to_csv('biofind_composite_data.csv')